In [1]:
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [83]:
#計測地点のosm_idを指定
start_osm = 204777946
mid_osm = []
end_osm = 1025226214
start_mesh = 50325693243
end_mesh = 50325674423

In [84]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [85]:

sttime = "2022-04-01 18:00:00"
dbname = "kddi_trkn_change"

In [86]:
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM kddi_18_param_copy  "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        sql += "ORDER BY trakanmap.gridcode"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, param) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                cur.execute(sql)
                connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, param) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        cur.execute(sql)
        connection.commit()
                
        cur.close()
        #print(lstItems)
        dte = dte + datetime.timedelta(minutes=15)

33.8140625000:33.8286458335:132.797656250:132.797656250
50325684314
存在メッシュ：50325684314値98.99060560941678距離0.8178571707903889 km
50325693222
存在メッシュ：50325693222値95.2103545753417距離0.37559142325038114 km
50325684333
存在メッシュ：50325684333値96.66038866576683距離0.5452363503146695 km
50325693243
存在メッシュ：50325693243値92.0距離0.0 km
50325693241
存在メッシュ：50325693241値92.98758231465644距離0.11554095924100155 km
50325684331
存在メッシュ：50325684331値97.52242256390373距離0.6460889293967952 km
50325684321
存在メッシュ：50325684321値100.49988142330868距離0.9944330092884479 km
50325693223
存在メッシュ：50325693223値93.97516398816342距離0.23108184347152202 km
50325684332
存在メッシュ：50325684332値98.17575213986419距離0.7225244070139163 km
50325693224
存在メッシュ：50325693224値94.33018666169654距離0.27261727767640936 km
50325684312
存在メッシュ：50325684312値99.8452441519745距離0.9178445394846149 km
50325683444
存在メッシュ：50325683444値96.13927771584096距離0.48426962568638426 km
50325694111
存在メッシュ：50325694111値95.85900549372252距離0.45147952716843076 km
50325684323
存在メッシュ：50325684323値

33.8140625000:33.8286458335:132.797656250:132.797656250
50325684314
存在メッシュ：50325684314値81.37363453233866距離0.8178571707903889 km
50325693222
50325693222 is kddi none!
50325684333
存在メッシュ：50325684333値71.91569517281832距離0.5452363503146695 km
50325693243
存在メッシュ：50325693243値53.0距離0.0 km
50325693241
存在メッシュ：50325693241値57.00842233595849距離0.11554095924100155 km
50325684331
存在メッシュ：50325684331値75.41453864172692距離0.6460889293967952 km
50325684321
存在メッシュ：50325684321値87.49951871813525距離0.9944330092884479 km
50325693223
存在メッシュ：50325693223値61.01684206960444距離0.23108184347152202 km
50325684332
存在メッシュ：50325684332値78.06628809709579距離0.7225244070139163 km
50325693224
存在メッシュ：50325693224値62.457816450415386距離0.27261727767640936 km
50325684312
存在メッシュ：50325684312値84.84246155801411距離0.9178445394846149 km
50325683444
50325683444 is kddi none!
50325694111
存在メッシュ：50325694111値68.66302229805021距離0.45147952716843076 km
50325684323
50325684323 is kddi none!
50325684141
50325684141 is kddi none!
50325684122
存在メッシュ：5032

33.8140625000:33.8286458335:132.797656250:132.797656250
50325684314
存在メッシュ：50325684314値50.27090797620614距離0.8178571707903889 km
50325693222
存在メッシュ：50325693222値40.931464244961845距離0.37559142325038114 km
50325684333
存在メッシュ：50325684333値44.51390140954158距離0.5452363503146695 km
50325693243
存在メッシュ：50325693243値33.0距離0.0 km
50325693241
存在メッシュ：50325693241値35.43990924797473距離0.11554095924100155 km
50325684331
存在メッシュ：50325684331値46.64363221670334距離0.6460889293967952 km
50325684321
存在メッシュ：50325684321値53.99970704582145距離0.9944330092884479 km
50325693223
50325693223 is kddi none!
50325684332
存在メッシュ：50325684332値48.25774058084092距離0.7225244070139163 km
50325693224
存在メッシュ：50325693224値38.75693175242676距離0.27261727767640936 km
50325684312
存在メッシュ：50325684312値52.38236790487815距離0.9178445394846149 km
50325683444
存在メッシュ：50325683444値43.226450827371806距離0.48426962568638426 km
50325694111
存在メッシュ：50325694111値42.53401357272621距離0.45147952716843076 km
50325684323
50325684323 is kddi none!
50325684141
存在メッシュ：503256

33.8140625000:33.8286458335:132.797656250:132.797656250
50325684314
50325684314 is kddi none!
50325693222
存在メッシュ：50325693222値24.74261985817704距離0.37559142325038114 km
50325684333
50325684333 is kddi none!
50325693243
存在メッシュ：50325693243値17.0距離0.0 km
50325693241
存在メッシュ：50325693241値19.381816170642004距離0.11554095924100155 km
50325684331
存在メッシュ：50325684331値30.318783830591354距離0.6460889293967952 km
50325684321
存在メッシュ：50325684321値37.49971402092094距離0.9944330092884479 km
50325693223
存在メッシュ：50325693223値21.763630794982348距離0.23108184347152202 km
50325684332
存在メッシュ：50325684332値31.894461043201847距離0.7225244070139163 km
50325693224
存在メッシュ：50325693224値22.61986194879755距離0.27261727767640936 km
50325684312
存在メッシュ：50325684312値35.92088295476201距離0.9178445394846149 km
50325683444
存在メッシュ：50325683444値26.98296390291057距離0.48426962568638426 km
50325694111
50325694111 is kddi none!
50325684323
存在メッシュ：50325684323値35.61416271879126距離0.9029656622985835 km
50325684141
存在メッシュ：50325684141値41.53117991316882距離1.18999

33.8140625000:33.8286458335:132.797656250:132.797656250
50325684314
存在メッシュ：50325684314値12.700908852044172距離0.8178571707903889 km
50325693222
存在メッシュ：50325693222値10.699599481063252距離0.37559142325038114 km
50325684333
存在メッシュ：50325684333値11.46726458775891距離0.5452363503146695 km
50325693243
存在メッシュ：50325693243値9.0距離0.0 km
50325693241
存在メッシュ：50325693241値9.522837695994586距離0.11554095924100155 km
50325684331
存在メッシュ：50325684331値11.923635475007858距離0.6460889293967952 km
50325684321
存在メッシュ：50325684321値13.499937224104597距離0.9944330092884479 km
50325693223
50325693223 is kddi none!
50325684332
存在メッシュ：50325684332値12.269515838751625距離0.7225244070139163 km
50325693224
存在メッシュ：50325693224値10.233628232662877距離0.27261727767640936 km
50325684312
存在メッシュ：50325684312値13.153364551045318距離0.9178445394846149 km
50325683444
存在メッシュ：50325683444値11.191382320151101距離0.48426962568638426 km
50325694111
存在メッシュ：50325694111値11.04300290844133距離0.45147952716843076 km
50325684323
50325684323 is kddi none!
50325684141
存在メッシュ：5

...

33.8140625000:33.8286458335:132.797656250:132.797656250
50325684314
存在メッシュ：50325684314値18.579393514745195距離0.8178571707903889 km
50325693222
存在メッシュ：50325693222値15.021510188556894距離0.37559142325038114 km
50325684333
存在メッシュ：50325684333値16.386248156015842距離0.5452363503146695 km
50325693243
存在メッシュ：50325693243値12.0距離0.0 km
50325693241
存在メッシュ：50325693241値12.92948923732371距離0.11554095924100155 km
50325684331
存在メッシュ：50325684331値17.197574177791747距離0.6460889293967952 km
50325684321
存在メッシュ：50325684321値19.999888398408174距離0.9944330092884479 km
50325693223
存在メッシュ：50325693223値13.858977871212623距離0.23108184347152202 km
50325684332
存在メッシュ：50325684332値17.81247260222511距離0.7225244070139163 km
50325693224
存在メッシュ：50325693224値14.193116858067336距離0.27261727767640936 km
50325684312
存在メッシュ：50325684312値19.383759201858346距離0.9178445394846149 km
50325683444
存在メッシュ：50325683444値15.895790791379735距離0.48426962568638426 km
50325694111
50325694111 is kddi none!
50325684323
50325684323 is kddi none!
50325684141
存在メッシュ

33.8140625000:33.8286458335:132.797656250:132.797656250
50325684314
存在メッシュ：50325684314値65.85969588153456距離0.8178571707903889 km
50325693222
存在メッシュ：50325693222値56.74261985817704距離0.37559142325038114 km
50325684333
存在メッシュ：50325684333値60.239760899790596距離0.5452363503146695 km
50325693243
存在メッシュ：50325693243値49.0距離0.0 km
50325693241
存在メッシュ：50325693241値51.381816170642004距離0.11554095924100155 km
50325684331
存在メッシュ：50325684331値62.31878383059136距離0.6460889293967952 km
50325684321
存在メッシュ：50325684321値69.49971402092095距離0.9944330092884479 km
50325693223
存在メッシュ：50325693223値53.76363079498235距離0.23108184347152202 km
50325684332
存在メッシュ：50325684332値63.894461043201844距離0.7225244070139163 km
50325693224
存在メッシュ：50325693224値54.61986194879755距離0.27261727767640936 km
50325684312
存在メッシュ：50325684312値67.920882954762距離0.9178445394846149 km
50325683444
50325683444 is kddi none!
50325694111
存在メッシュ：50325694111値58.30701324956606距離0.45147952716843076 km
50325684323
存在メッシュ：50325684323値67.61416271879126距離0.902965662298

33.8140625000:33.8286458335:132.797656250:132.797656250
50325684314
存在メッシュ：50325684314値48.62605959751984距離0.8178571707903889 km
50325693222
存在メッシュ：50325693222値40.176086697822626距離0.37559142325038114 km
50325684333
存在メッシュ：50325684333値43.417339370537626距離0.5452363503146695 km
50325693243
存在メッシュ：50325693243値33.0距離0.0 km
50325693241
存在メッシュ：50325693241値35.207536938643806距離0.11554095924100155 km
50325684331
存在メッシュ：50325684331値45.3442386722554距離0.6460889293967952 km
50325684321
存在メッシュ：50325684321値51.999734946219405距離0.9944330092884479 km
50325693223
50325693223 is kddi none!
50325684332
50325684332 is kddi none!
50325693224
存在メッシュ：50325693224値38.20865253790992距離0.27261727767640936 km
50325684312
存在メッシュ：50325684312値50.536428104413574距離0.9178445394846149 km
50325683444
50325683444 is kddi none!
50325694111
存在メッシュ：50325694111値41.62601228008562距離0.45147952716843076 km
50325684323
50325684323 is kddi none!
50325684141
存在メッシュ：50325684141値55.736215529278425距離1.1899978232648964 km
50325684122
存在メッシュ：

50325684314
50325684314 is kddi none!
50325693222
存在メッシュ：50325693222値31.510755094278448距離0.37559142325038114 km
50325684333
存在メッシュ：50325684333値32.19312407800792距離0.5452363503146695 km
50325693243
存在メッシュ：50325693243値30.0距離0.0 km
50325693241
存在メッシュ：50325693241値30.464744618661854距離0.11554095924100155 km
50325684331
存在メッシュ：50325684331値32.598787088895875距離0.6460889293967952 km
50325684321
存在メッシュ：50325684321値33.99994419920409距離0.9944330092884479 km
50325693223
存在メッシュ：50325693223値30.92948893560631距離0.23108184347152202 km
50325684332
50325684332 is kddi none!
50325693224
存在メッシュ：50325693224値31.096558429033667距離0.27261727767640936 km
50325684312
存在メッシュ：50325684312値33.69187960092917距離0.9178445394846149 km
50325683444
存在メッシュ：50325683444値31.94789539568987距離0.48426962568638426 km
50325694111
50325694111 is kddi none!
50325684323
存在メッシュ：50325684323値33.63203175000805距離0.9029656622985835 km
50325684141
存在メッシュ：50325684141値34.7865716903744距離1.1899978232648964 km
50325684122
存在メッシュ：50325684122値35.88197465

50325684314
存在メッシュ：50325684314値24.700908852044172距離0.8178571707903889 km
50325693222
50325693222 is kddi none!
50325684333
存在メッシュ：50325684333値23.467264587758912距離0.5452363503146695 km
50325693243
50325693243 is kddi none!
50325693241
存在メッシュ：50325693241値21.522837695994586距離0.11554095924100155 km
50325684331
存在メッシュ：50325684331値23.92363547500786距離0.6460889293967952 km
50325684321
存在メッシュ：50325684321値25.4999372241046距離0.9944330092884479 km
50325693223
50325693223 is kddi none!
50325684332
50325684332 is kddi none!
50325693224
存在メッシュ：50325693224値22.233628232662877距離0.27261727767640936 km
50325684312
存在メッシュ：50325684312値25.153364551045318距離0.9178445394846149 km
50325683444
存在メッシュ：50325683444値23.1913823201511距離0.48426962568638426 km
50325694111
存在メッシュ：50325694111値23.04300290844133距離0.45147952716843076 km
50325684323
存在メッシュ：50325684323値25.086035718759057距離0.9029656622985835 km
50325684141
存在メッシュ：50325684141値26.384893151671204距離1.1899978232648964 km
50325684122
存在メッシュ：50325684122値27.6172214848148

In [87]:
connection.close()

In [ ]:
#CreateGeojsonFile